In [ ]:
!pip install arcgis==2.0.0 isodate

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import pandas as pd
from arcgis.gis import GIS
import requests
import numpy as np
from datetime import datetime, timedelta
import isodate
import os

import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

PORTAL = "portal"
IMAGE_SERVER = "image"
VECTOR_SERVER = "server"
PRODUCTION_HOST = "maps.water.noaa.gov"
STAGING_HOST = "maps-staging.water.noaa.gov"
TESTING_HOST = "maps-testing.water.noaa.gov"

suffix_mapper = {"testing": "_alpha", "staging": "_beta", "production": ""}
priority_services = r"helper_files/hydrovis_services.csv"
column_mapper = {'testing': 'testing_rest', 'staging': 'staging_rest', 'production': 'prod_rest'}
host_mapper = {'testing': TESTING_HOST, 'staging': STAGING_HOST, 'production': PRODUCTION_HOST}
rest_folders = ['nwm', 'rfc', 'owp']

def check_service_status(environment, tiers_to_check, display_portal_items=False, check_map_render=True):
    print(f"Connecting to {environment} portal")
    
    portal = f'https://{host_mapper[environment]}/portal'
    gis = GIS(portal, verify_cert=False, username="hydrovis.proc", password="WaterIsFun1@")

    df_status = pd.DataFrame()

#     df_status = df_status[~df_status.index.str.contains('retrospective')]
#     df_status = df_status[~df_status.index.str.contains('Sample')]
#     df_status.index = df_status.index.str.replace("\xa0", "")

    column = column_mapper[environment]

    tier_services = {}
    
    services = pd.read_csv(priority_services)
    
    df_status = services[["service_name", "tier", column_mapper[environment], "acceptable_delay"]]
    df_status = df_status.rename(columns={column_mapper[environment]: "rest"})
    
    df_status['portal_check'] = "unchecked"
    df_status['rest_check'] = "unchecked"
    
    if check_map_render:
        map1 = gis.map("Test Map")
        df_status['map_check'] = "unchecked"

    df_status['server'] = None
    df_status['reference_time'] = None
    df_status['update_time'] = None
    df_status['update_status'] = None
    
    df_status = df_status[df_status['tier'].isin(tiers_to_check)]
    
    token = gis._con.token
    df_status = df_status.set_index("service_name")

    print("Getting status for each tier service")
    
    for service in df_status.iterrows():
        row = service[1]
        base_service_name = service[0]
        full_service_name = f"{base_service_name}{suffix_mapper[environment]}"
        acceptable_delay = row["acceptable_delay"]
        rest_url = row["rest"]
        print(f"Checking {full_service_name}")
        item = None
        try:
            item = [resource for resource in gis.content.search(full_service_name, item_type='Map Service', max_items=10) if f"{full_service_name}/MapServer" in resource.url][0]
            df_status.loc[base_service_name, "portal_check"] = "Pass"
        except:
            try:
                item = [resource for resource in gis.content.search(full_service_name, item_type='Image Service', max_items=10) if f"{full_service_name}/ImageServer" in resource.url][0]
                df_status.loc[base_service_name, "portal_check"] = "Pass"
            except:
                print(f"{full_service_name} does not exist in the portal")
                df_status.loc[base_service_name, "portal_check"] = "Fail"

                if df_status.loc[base_service_name, "rest"]:
                    rest_url = df_status.loc[full_service_name, "rest"]
                else:
                    continue

        if display_portal_items and item:
            display(item)

        if '/server/' in rest_url:
            df_status.loc[base_service_name, "server"] = "server"
        else:
            df_status.loc[base_service_name, "server"] = "image"

        params = {
            'token': token,
            'f': 'json'
        }

        # TODO: Check REST not as admin
        query_url = f"{rest_url}/0/query?where=oid>=0&outFields=*&returnGeometry=false&resultRecordCount=1&f=json"
        
        if base_service_name == "rfc_5day_max_downstream_streamflow":
            query_url = f"{rest_url}/1/query?where=oid>=0&outFields=*&returnGeometry=false&resultRecordCount=1&f=json"
        
        if environment == "testing":
            query_url = query_url.replace(r"/nwm/", r"/NWM/")

        tries = 0
        ref_time = None
        while tries<3:
            tries += 1
            res = requests.get(query_url, params=params, verify=False).json()

            if 'error' in res:
                print(f"{base_service_name} does not exist in the rest - {rest_url} ({res})")
                df_status.loc[base_service_name, "rest_check"] = "Fail"
                if "Failed to execute query" not in res['error']['message']:
                    break
                print(f"Attempting {base_service_name} again - {tries} out of 3")
            else: 
                tries = 3
                
                df_status.loc[base_service_name, "rest_check"] = "Pass"
                
                try:
                    attributes = res['features'][0]['attributes']
                except:
                    try:
                        attributes = res['attributes']
                    except:
                        print(f"Failed to get data for {full_service_name} - {res}")
                        continue

                update_time = attributes.get("update_time")
                try:
                    update_time = datetime.strptime(update_time, "%Y-%m-%d %H:%M:%S UTC")
                except:
                    update_time = datetime.strptime(update_time, "%Y-%m-%d %H:%M UTC")
                df_status.loc[base_service_name, "update_time"] = update_time

                if base_service_name == "rfc_max_stage":
                    df_status.loc[base_service_name, "reference_time"] = "Pass"
                    ref_time = update_time
                else:
                    possible_times = ["ref_time", "Ref_Time", "Ref_time", "reference_time", "valid_time", "Valid_Time", "Valid_time"]

                    for possible_time in possible_times:
                        ref_time = attributes.get(possible_time)
                        if ref_time:
                            try:
                                ref_time = datetime.strptime(ref_time, "%Y-%m-%d %H:%M:%S UTC")
                            except:
                                ref_time = datetime.strptime(ref_time, "%Y-%m-%d %H:%M UTC")
                            df_status.loc[base_service_name, "reference_time"] = ref_time
                            break

        if ref_time and acceptable_delay:
            acceptable_delay = isodate.parse_duration(acceptable_delay)
            issue_delay = acceptable_delay + timedelta(hours=1)

            acceptable_threshold = ref_time + acceptable_delay
            issue_threshold = ref_time + issue_delay
            now = datetime.utcnow()

            if now <= acceptable_threshold:
                update_status = "GOOD"
            elif now <= issue_threshold:
                update_status = "DELAYED"
            else:
                update_status = "ISSUE"

            df_status.loc[base_service_name, "update_status"] = update_status

        if check_map_render and item:
            try:
                map1.add_layer(item)
                map1.remove_layers(item)
                df_status.loc[base_service_name, "map_check"] = "Pass"
            except Exception as e:
                print(f"{full_service_name} failed to load in a map - {e}")
                df_status.loc[base_service_name, "map_check"] = "Fail"

    df_status = df_status.rename_axis('Service Name').sort_values(["tier", "server", "Service Name"])
    df_status = df_status[["tier", "server", "portal_check", "rest_check", "map_check", "reference_time", "update_time", "update_status", "rest"]]
    
    df_formatted = df_status.style.format(make_clickable).applymap(bold_cells).applymap(color_cells)

    display(df_formatted)

def color_cells(val):
    if val in ['Pass', 'STARTED', 'GOOD']:
        color = 'green'
    elif val in ['Fail', 'STOPPED', np.nan, "ISSUE"]:
        color = 'red'
    elif val in ['DELAYED']:
        color = 'yellow'
    elif val in ['unchecked']:
        color = 'orange'
    else:
        color = 'white'
        
    return f'background-color: {color}'

def bold_cells(val):
    return 'font-weight: bold'

def make_clickable(val):
    if isinstance(val, str):
        if 'http' in val:
            return '<a target="_blank" href="{}">{}</a>'.format(val,val)
        
    return val

In [ ]:
tiers_to_check = [1]
environment = 'production'

check_service_status(environment, tiers_to_check)